# Assignment 6

## Group members

- Erdem Halil - gushaliler@student.gu.se

## Problem 1

#### Implement Dijkstra's and Shortest path in DAG algorithms

In [259]:
import math

class Graph:
    def __init__(self, gdict={}):
        self.gdict = gdict
        self.colour = {}
        self.distance = {}
        self.predecessor = {}
        self.finish = {}

    def get_vertices(self):
        return list(self.gdict.keys())

    def get_edges(self):
        edges = []
        for vertex in self.gdict:
            for next_vertex in self.gdict[vertex]:
                # use tuple instead of set
                if (vertex, next_vertex) not in edges:
                    edges.append((vertex, next_vertex))
        return edges

    def print_path(self, s, v):
        if v in self.gdict.keys():
            if v == s:
                print(s)
            elif self.predecessor[v] == None:
                print("There is no path from", s, "to", v, "exists.")
            else:
                self.print_path(s, self.predecessor[v])
                print(v)
        else:
            print("Node with key", v, "is not in the graph.")

    # initialise_single_source runs in O(V)
    def initialise_single_source(self, s):
        for v in self.get_vertices():
            self.distance[v] = math.inf
            self.predecessor[v] = None
        self.distance[s] = 0

    def get_weight(self, u, v):
        return self.gdict[u][v]

    def relax(self, u, v):
        if self.distance[v] > self.distance[u] + self.get_weight(u, v):
            self.distance[v] = self.distance[u] + self.get_weight(u, v)
            self.predecessor[v] = u

    def shortest_path_dijkstra(self, s, v):
        # Initialize the distances and predecessors
        self.initialise_single_source(s)
        # Set to keep track of visited vertices
        visited = set()
        # Priority queue (implemented as a list of tuples)
        queue = [(0, s)]

        # While the queue is not empty
        while queue:
            # Get the vertex with the minimum distance from the queue
            distance_u, u = min(queue)
            # Remove the vertex from the queue
            queue.remove((distance_u, u))
            # Mark the vertex as visited
            visited.add(u)

            # Relax edges from the current vertex to its neighbors
            for neighbor in self.gdict[u]:
                if neighbor not in visited:
                    self.relax(u, neighbor)
                    # Add the neighbor to the queue with updated distance
                    queue.append((self.distance[neighbor], neighbor))

        print(f'Shortest path from "{s}" to "{v}" takes {self.distance[v]} minutes')
        print("Path:")
        self.print_path(s, v)

    def shortest_path_dag(self, s, v):
        # Set to keep track of visited vertices
        visited = set()
        # List to store the topological order
        topological_order = []

        def visit(vertex):
            # Mark the current vertex as visited
            visited.add(vertex)

            # Visit all neighbours
            for neighbor in self.gdict[vertex]:
                if neighbor not in visited:
                    visit(neighbor)
            # Add the current vertex to the list
            topological_order.append(vertex)

        # Go through all vertices in the graph
        for vertex in self.get_vertices():
            if vertex not in visited:
                # DFS from all unvisited vertices
                visit(vertex)
        
        # Return the reverse of the list to get the topological order
        topological_order = topological_order[::-1]
        # Initialize the distances and predecessors
        self.initialise_single_source(s)

        # Relax edges in topological order
        for node in topological_order:
            for neighbor in self.gdict[node]:
                self.relax(node, neighbor)

        print(f'Shortest path from "{s}" to "{v}" takes {self.distance[v]} minutes')
        print("Path:")
        self.print_path(s, v)

# Input graphs
adjacency = {
    "r": {"s": 5, "t": 3},
    "s": {"x": 6, "t": 2},
    "t": {"x": 7, "y": 4, "z": 2},
    "x": {"y": -1, "z": 1},
    "y": {"z": -2},
    "z": {}
}

adjacency2 = {
    "s": {"t": 5, "y": 10},
    "t": {"x": 1, "y": 2},
    "x": {"z": 4},
    "y": {"x": 9,"t": 3, "z": 2},
    "z": {"s": 7,"x": 6}
}

graph = Graph(adjacency2)
graph2 = Graph(adjacency)
graph.shortest_path_dijkstra("s", "z") # This is just a method to print out the output (total path).
graph2.shortest_path_dag("s", "z") # This is just a method to print out the output (total path).

Shortest path from "s" to "z" takes 9 minutes
Path:
s
t
y
z
Shortest path from "s" to "z" takes 3 minutes
Path:
s
x
y
z


## Problem 2

#### Put all the data about the tram lines, in both directions, into a data structure suitable for route finding purposes

In [260]:
import tarfile

def tar_to_dict(filepath: str):
    tram_network = {}   # Dictionary to build the tram network
    terminals = set()   # Set to keep track of terminal stations (uniqueness is important)
    with tarfile.open(filepath, "r:*") as tar:
        for file in tar:
            if file.name.startswith("Data/tram"):
                txt_file = tar.extractfile(file)
                lines = txt_file.read().splitlines()

                # Go through each line in each file
                for i in range(len(lines) - 1):
                    # Get the stopname and the time it takes to reach the next stop
                    stop, time = lines[i].decode('utf-8', errors='ignore').strip().lower().split(",")
                    time = int(time)
                    # Get the next stopname, omit next time as it's not useful
                    next_stop, _ = lines[i + 1].decode('utf-8', errors='ignore').strip().lower().split(",")

                    # If the stop is not in the dictionary yet, add it and attach it to an empty dict as value
                    if tram_network.get(stop) is None:
                        tram_network[stop] = {}
                    
                    # Check if the next stop is already attached to the current stop
                    value = tram_network[stop].get(next_stop)
                    if value is None or value > time:
                        # If it's not or if the existing time is higher than current reading, add/update
                        tram_network[stop][next_stop] = time

                    # If the next stop is not in the dictionary yet, add it and attach it to an empty dict as value
                    # This is for the reverse routes 
                    if tram_network.get(next_stop) is None:
                        tram_network[next_stop] = {}

                    # Check if the current stop is already attached to the next stop
                    value = tram_network[next_stop].get(stop)
                    if value is None or value > time:
                        # If it's not or if the existing time is higher than current reading, add/update
                        tram_network[next_stop][stop] = time

                    # If it's the first or the last stop of the tramline, add it to the set
                    if i == 0:  terminals.add(stop)
                    if i == len(lines) - 2: terminals.add(next_stop)

    return tram_network, terminals

tram_network, terminal_stops = tar_to_dict("Data_A6.tar.gz")
print(tram_network)
print(terminal_stops)
print(f"Number of terminal stops: {len(terminal_stops)}")

{'opaltorget': {'smaragdgatan': 1}, 'smaragdgatan': {'opaltorget': 1, 'briljantgatan': 1}, 'briljantgatan': {'smaragdgatan': 1, 'frölunda torg spårvagn': 1}, 'frölunda torg spårvagn': {'briljantgatan': 1, 'positivgatan': 1}, 'positivgatan': {'frölunda torg spårvagn': 1, 'musikvägen': 1}, 'musikvägen': {'positivgatan': 1, 'nymilsgatan': 1}, 'nymilsgatan': {'musikvägen': 1, 'lantmilsgatan': 1}, 'lantmilsgatan': {'nymilsgatan': 1, 'axel dahlströms torg': 2}, 'axel dahlströms torg': {'lantmilsgatan': 2, 'marklandsgatan': 1}, 'marklandsgatan': {'axel dahlströms torg': 1, 'botaniska trädgården': 3, 'bokekullsgatan': 1, 'mariaplan': 5, 'botaniska trädgård': 3}, 'botaniska trädgården': {'marklandsgatan': 3, 'linnéplatsen': 2, 'sahlgrenska huvudentré': 3}, 'linnéplatsen': {'botaniska trädgården': 2, 'olivedalsgatan': 1, 'sahlgrenska huvudentré': 3}, 'olivedalsgatan': {'linnéplatsen': 1, 'prinsgatan': 2, 'seminariegatan': 2}, 'prinsgatan': {'olivedalsgatan': 2, 'järntorget': 1}, 'järntorget': {'

#### Given the result from the previous step, write Python code that finds the tram-hubs. A tram-hub is defined as a tram stop that is directly connected to atleast three others.

In [261]:
def get_tram_hubs(tram_dict: dict):
    tram_hubs = {}

    # Go through the tram network and extract those connected to at least 3 more
    for k, v in tram_dict.items():
        if len(v) >= 3:
            tram_hubs[k] = v
            
    return tram_hubs

tram_hubs = get_tram_hubs(tram_network)
print(tram_hubs)
print(f"Number of tramhubs: {len(tram_hubs)}")

{'marklandsgatan': {'axel dahlströms torg': 1, 'botaniska trädgården': 3, 'bokekullsgatan': 1, 'mariaplan': 5, 'botaniska trädgård': 3}, 'botaniska trädgården': {'marklandsgatan': 3, 'linnéplatsen': 2, 'sahlgrenska huvudentré': 3}, 'linnéplatsen': {'botaniska trädgården': 2, 'olivedalsgatan': 1, 'sahlgrenska huvudentré': 3}, 'olivedalsgatan': {'linnéplatsen': 1, 'prinsgatan': 2, 'seminariegatan': 2}, 'järntorget': {'prinsgatan': 1, 'stenpiren': 2, 'masthuggstorget': 2, 'hagakyrkan': 3}, 'brunnsparken': {'stenpiren': 3, 'centralstationen': 2, 'kungsportsplatsen': 1, 'domkyrkan': 2, 'nordstan': 2, 'lilla bommen': 0}, 'centralstationen': {'brunnsparken': 2, 'ullevi norra': 3, 'ullevi södra': 3, 'gamlestads torg': 6}, 'ullevi norra': {'centralstationen': 3, 'svingeln': 2, 'ullevi södra': 2}, 'redbergsplatsen': {'olskrokstorget': 1, 'stockholmsgatan': 1, 'ejdergatan': 1}, 'härlanda': {'stockholmsgatan': 1, 'munkebäckstorget': 2, 'solrosgatan': 2}, 'munkebäckstorget': {'härlanda': 2, 'ättehö

####  Write Python code that creates a simplified graph of the tram network whose nodes are the tram-hubs found in the previous step and the terminal stops of each tram line (that is, all non-hub tram stops and non-terminal tram stops should be omitted).

In [262]:
def get_terminal_stops(tram_dict: dict, terminal_list: list):
    terminals = {}

    # Go through the tram network and extract all terminal stations
    for k, v in tram_dict.items():
        if k in terminal_list:
            terminals[k] = v

    return terminals

terminal_stops = get_terminal_stops(tram_network, terminal_stops)
print(terminal_stops)

{'opaltorget': {'smaragdgatan': 1}, 'frölunda torg spårvagn': {'briljantgatan': 1, 'positivgatan': 1}, 'axel dahlströms torg': {'lantmilsgatan': 2, 'marklandsgatan': 1}, 'marklandsgatan': {'axel dahlströms torg': 1, 'botaniska trädgården': 3, 'bokekullsgatan': 1, 'mariaplan': 5, 'botaniska trädgård': 3}, 'centralstationen': {'brunnsparken': 2, 'ullevi norra': 3, 'ullevi södra': 3, 'gamlestads torg': 6}, 'östra sjukhuset': {'tingvallsvägen': 1}, 'virginsgatan': {'sanatoriegatan': 1, 'welandergatan': 1}, 'mölndals innerstad': {'mölndals sjukhus': 1}, 'varmfrontsgatan': {'temperaturgatan': 1}, 'väderilsgatan': {'temperaturgatan': 1, 'friskväderstorget': 1}, 'aprilgatan': {'allhelgonakyrkan': 1}, 'komettorget': {'rymdtorget': 1, 'rymdtorget spårvagn': 1}, 'angered centrum': {'storås': 3}, 'saltholmen': {'roddföreningen': 1}, 'kungssten': {'nya varvsallén': 1, 'sandarna': 2}, 'doktor sydows gata': {'doktor fries torg': 2}}


In [263]:
# Combine all hubs and terminals by omitting reoccuring ones
final_network = tram_hubs | terminal_stops

print(final_network)
print(f"Number of stops in the scaled down network: {len(final_network)}")
# The reason why it's 46 is that 2 of the stops are both terminals and hubs

{'marklandsgatan': {'axel dahlströms torg': 1, 'botaniska trädgården': 3, 'bokekullsgatan': 1, 'mariaplan': 5, 'botaniska trädgård': 3}, 'botaniska trädgården': {'marklandsgatan': 3, 'linnéplatsen': 2, 'sahlgrenska huvudentré': 3}, 'linnéplatsen': {'botaniska trädgården': 2, 'olivedalsgatan': 1, 'sahlgrenska huvudentré': 3}, 'olivedalsgatan': {'linnéplatsen': 1, 'prinsgatan': 2, 'seminariegatan': 2}, 'järntorget': {'prinsgatan': 1, 'stenpiren': 2, 'masthuggstorget': 2, 'hagakyrkan': 3}, 'brunnsparken': {'stenpiren': 3, 'centralstationen': 2, 'kungsportsplatsen': 1, 'domkyrkan': 2, 'nordstan': 2, 'lilla bommen': 0}, 'centralstationen': {'brunnsparken': 2, 'ullevi norra': 3, 'ullevi södra': 3, 'gamlestads torg': 6}, 'ullevi norra': {'centralstationen': 3, 'svingeln': 2, 'ullevi södra': 2}, 'redbergsplatsen': {'olskrokstorget': 1, 'stockholmsgatan': 1, 'ejdergatan': 1}, 'härlanda': {'stockholmsgatan': 1, 'munkebäckstorget': 2, 'solrosgatan': 2}, 'munkebäckstorget': {'härlanda': 2, 'ättehö

#### Test the algorithms you implemented in Problem 1 on this data.

In [264]:
# Test on the entire network
graph = Graph(tram_network)

print("Chalmers to Centralstationen using Dijkstra's:")
graph.shortest_path_dijkstra("chalmers", "centralstationen")

print("\nChalmers to Centralstationen using DAG shortest path:")
graph.shortest_path_dag("chalmers", "centralstationen")

print("\nSaltholmen to Chalmers using Dijkstra's:")
graph.shortest_path_dijkstra("saltholmen", "chalmers")

print("\nSaltholmen to Chalmers using DAG shortest path:")
graph.shortest_path_dag("saltholmen", "chalmers")

Chalmers to Centralstationen using Dijkstra's:
Shortest path from "chalmers" to "centralstationen" takes 10 minutes
Path:
chalmers
kapellplatsen
vasaplatsen
grönsakstorget
domkyrkan
brunnsparken
centralstationen

Chalmers to Centralstationen using DAG shortest path:
Shortest path from "chalmers" to "centralstationen" takes inf minutes
Path:
There is no path from chalmers to centralstationen exists.

Saltholmen to Chalmers using Dijkstra's:
Shortest path from "saltholmen" to "chalmers" takes 28 minutes
Path:
saltholmen
roddföreningen
långedrag
hinsholmen
käringberget
tranered
hagen
nya varvsallén
kungssten
sandarna
sannaplan
mariaplan
marklandsgatan
botaniska trädgård
sahlgrenska huvud
medicinaregatan
wavrinskys plats
chalmers

Saltholmen to Chalmers using DAG shortest path:
Shortest path from "saltholmen" to "chalmers" takes inf minutes
Path:
There is no path from saltholmen to chalmers exists.


In [265]:
# Test on the final network where only hubs and terminals are considered
graph = Graph(final_network)

print("\nChalmers to Centralstationen using Dijkstra's:")
graph.shortest_path_dijkstra("chalmers", "centralstationen")

print("\nChalmers to Centralstationen using DAG shortest path:")
graph.shortest_path_dag("chalmers", "centralstationen")

print("\nSaltholmen to Chalmers using Dijkstra's:")
graph.shortest_path_dijkstra("saltholmen", "chalmers")

print("\nSaltholmen to Chalmers using DAG shortest path:")
graph.shortest_path_dag("saltholmen", "chalmers")


Chalmers to Centralstationen using Dijkstra's:


KeyError: 'kapellplatsen'

#### Observations

- Casing for some of the stop names was inconsistent so I've converted every name to lowercase.
    - There is a possible edge case that I haven't addressed - `botaniska trädgården` and `botaniska trädgård` appear for `marklandsgatan`. I am not sure if those are two different stops or it's just a naming error.
- For some of the stops, some of the trams have different travel times although it's the exact same route. As a result, I've decided to keep the lowest travel time.
- As per the assignment description: `(the last line of a file contains the name of the terminal tram stop for that tram line, and the value “0”)`. Judging by this statement a tram would have only one terminal stop when in fact it should be two as trams go both ways. I've considered both the first and the last stops to be a terminal one. 


I have presented results with both complete and scaled down network. I anticipated that DAG shortest part algorithm may not work as the graph built by the network data could feature loops (cycles) and as the name suggests, DAG shortest path algorithm is built for Directed Acyclic Graphs. I have been unable to figure out why both algorithms do not work on the network with just terminals and tramhubs. I would really appreciate some input on the matter.